<a href="https://colab.research.google.com/github/DuanMingbai/PyTorch_Tutorials_cn/blob/main/Introduction%20to%20PyTorch/7_%E4%BC%98%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#优化模型参数

现在我们有了模型和数据，是时候通过在我们的数据基础上优化参数来训练，验证和测试我们的模型了。训练模型是一个迭代的过程，在每一次迭代（我们称其为*epoch*）中模型会对输出做一个预测，计算预测的误差（loss），收集误差关于其参数的导数（我们在前一章讲到的），并且使用梯度下降优化这些**参数**，有关这一步更详细的步骤，请观看这个视频https://www.youtube.com/watch?v=tIeHLnjs5U8。

##之前的代码

我们从之前讲的Datasets & DataLoaders和创建模型中获取代码。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



##超参数

超参数是一个可以调整的参数，让你控制模型优化过程。不同的超参数的值可以影响模型的训练和收敛速率（[阅读更多](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)关于模型超参数的调整）

\\
我们在训练的过程中定义如下的超参数：


*   Number of Epochs-数据及循环的次数
*   Batch Size-在参数更新前，经过神经网络的数据样本的数量
*   Learning Rate-在每一个epoch/batch中应该把参数更新多少，在训练过程中，小的lr会导致缓慢的学习速率，大的lr会导致不可预测的行为。



In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

##优化循环

一旦我们设置了超参数，我们可以训练并优化我们的模型通过一个优化循环。每一个优化循环的迭代被称作**epoch**。

\\
每一个epoch由两个主要部分组成：


*   训练循环-在训练集上迭代，并且尝试通过收敛找到最优参数
*   验证循环-在验证集上迭代，并检查模型性能是否提高


\\

\\
让我们简要熟悉一下训练循环中使用的一些概念。跳转到[这里](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#full-impl-label)看循环的完整实现



##损失函数

当呈现一些训练数据时，我们未经训练的网络可能不会给出正确的答案。**损失函数**测量目标值和获取的结果之间的不同程度，并且在训练过程中我们希望最小化损失函数。使用我们给定的数据样本作为输入并将其与真是数据的标签比较来计算loss。

\\
通常的损失函数包含对于回归任务的nn.MSELoss(Mean Square Error)，对于分类任务的nn.NLLLoss（负对数似然）。nn.CrossEntropyLoss结合了```nn.LogSoftmax```和```nn.NLLLoss```。

\\
我们将模型输出的logits送到nn.CrossEntropyLoss中，这样可以对logits进行归一化并且计算预测误差。

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

##优化器

优化在每一个训练的步骤中，通过调整模型参数去减少误差的过程。优化算法定义了这个过程如何被实现（在这个例子中我们是使用随机梯度下降）。所有优化的逻辑被封装在```optimizer```这个类中。在这里，我们使用SGD优化器；此外，PyTorch还有许多不同的优化器，例如ADAM和RMSProp，对于不同类型的模型和数据效果更好。

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化发生在一下三步



*   调用optimizer.zero_grad()去重置模型的参数，梯度默认被加起来防止重复计算，我们会在每次迭代的时候将他们归0
*   通过调用```loss.backward()```函数来反向传播预测的loss，PyTorch存储每个参数的wrt的loss。
*   一旦我们有了梯度，我们调用```optimizer.step()```通过在反向传播中收集的梯度来调整参数。


##整个的操作

我们定义```train_loop```来循环我们的优化代码，用```test_loop```根据测试数据来评估模型的效果

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化loss函数和优化器，并把他们送进```train_loop```和```test_loop```，任意的增加模型迭代的次数来增加模型性能的改进。

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314211  [    0/60000]
loss: 2.293885  [ 6400/60000]
loss: 2.274530  [12800/60000]
loss: 2.259786  [19200/60000]
loss: 2.245999  [25600/60000]
loss: 2.219560  [32000/60000]
loss: 2.226327  [38400/60000]
loss: 2.194648  [44800/60000]
loss: 2.202966  [51200/60000]
loss: 2.158596  [57600/60000]
Test Error: 
 Accuracy: 41.9%, Avg loss: 2.156872 

Epoch 2
-------------------------------
loss: 2.177207  [    0/60000]
loss: 2.163566  [ 6400/60000]
loss: 2.108867  [12800/60000]
loss: 2.118869  [19200/60000]
loss: 2.068160  [25600/60000]
loss: 2.008330  [32000/60000]
loss: 2.041187  [38400/60000]
loss: 1.961023  [44800/60000]
loss: 1.986273  [51200/60000]
loss: 1.900053  [57600/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 1.901964 

Epoch 3
-------------------------------
loss: 1.941878  [    0/60000]
loss: 1.911184  [ 6400/60000]
loss: 1.800246  [12800/60000]
loss: 1.836955  [19200/60000]
loss: 1.715138  [25600/60000]
loss: 1.669181  [32000/600

##更多阅读

\\


*   [Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
*   [torch.optim](https://pytorch.org/docs/stable/optim.html)
*   [Warmstart Training a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)

